# Testing notebook for all Model Readers

In [1]:
#read in information for testing
from csv import reader
reader_csv = 'C:/Users/rringuet/Kamodo_Dev/ReaderMapping.csv'
read_obj = open(reader_csv, 'r')
csv_reader = reader(read_obj, delimiter=',')
reader_dict = {}
print(next(csv_reader))
for row in csv_reader: 
    if row[0]=='': continue  #ignore empty rows
    reader_dict[row[0]] = row[1:]
read_obj.close()
for key in reader_dict.keys(): print('\n',key, ':', reader_dict[key])

['ï»¿#Model', 'reader_name', 'file_prefix', 'run_name', 'variable_name', 'filetimes_start', 'filetimes_stop', 'datetime_start', 'datetime_stop', 'time_hrs', 'c1', 'c2', 'c3', 'interpolated_value', 'file_dir', 'start_utc', 'end_utc', 'satellite', 'output_name', 'plot_output']

 CTIPe : ['ctipe_4D', 'C:/Users/rringuet/Kamodo_WinDev1/CTIPe/Data/2015-03-20', 'Sujay_Pal_042620_IT_1', 'T_n', '1426809600', '1426896000', '2015-03-20 00:00:00', '2015-03-21 00:00:00', '23.5', '331.5', '-85.5', '1.019', '195.2375037', 'C:/Users/rringuet/Kamodo_WinDev1/CTIPe/Data/', '1426660000', '1426880700', 'grace1', 'C:/Users/rringuet/Kamodo_WinDev1/CTIPe', 'C:/Users/rringuet/Kamodo_WinDev1/CTIPe']

 IRI : ['iri_4D', 'C:/Users/rringuet/Kamodo_WinDev1/IRI/Data/IRI.3D.2017148.nc', 'Ola_Abuelezz_011121_IT_12_doy_147', 'T_n', '1495929600', '1496015100', '2017-05-28 00:00:00', '2017-05-28 23:45:00', '23.5', '331.5', '-85.5', '1.019', '370.5292023', 'C:/Users/rringuet/Kamodo_WinDev1/IRI/Data/', '1495930160', '149601

In [2]:
#define testing functions to be run in a loop
def test_modelname(kamodo_object, model):
    #compare model name attribute with stored name
    assert kamodo_object.modelname == model, f'kamodo_object.modelname likely not assigned.'

def test_all_variables(module):
    #check for existence of model_varnames dictionary 
    assert isinstance(module.model_varnames, (dict)), 'model_varnames not a dict'
    
def test_file_variables(kamodo_object):
    #check for correct output type 
    assert isinstance(kamodo_object.var_dict, (dict)), 'kamodo_object.var_dict is not a dict'
    
def test_file_varname(kamodo_object, varname):
    #check that chosen variable is in file
    assert varname in kamodo_object.var_dict.keys(), f'{varname} is not in kamodo_object.var_dict'
        
def test_utcstart(kamodo_object, utc_start):
    #compare utc start timestamp to stored value
    assert int(utc_start) == int(kamodo_object.filetimes[0]), f'{int(utc_start)} != {int(kamodo_object.filetimes[0])}'
    
def test_utcstop(kamodo_object, utc_stop):
    #compare utc stop timestamp to stored value
    assert int(utc_stop) == int(kamodo_object.filetimes[1]), f'{int(utc_stop)} != {int(kamodo_object.filetimes[1])}'
    
def test_startstr(kamodo_object, start_str):
    #compare utc start time string to stored value
    assert start_str == kamodo_object.datetimes[0], f'{start_str} != {kamodo_object.datetimes[0]}'
        
def test_stopstr(kamodo_object, stop_str):
    #compare utc stop time string to stored value
    assert stop_str == kamodo_object.datetimes[1], f'{stop_str} != {kamodo_object.datetimes[1]}'
    
def test_datetime(kamodo_object):
    #test that ko.filedate is a datetime object
    import datetime
    assert isinstance(kamodo_object.filedate, (datetime.datetime)), f'{kamodo_object.filedate} is not a datetime object.'

def test_variablecoordinates(kamodo_object, varname):
    #test that requested variable coordinate dependencies were properly stored in xvec 
    assert isinstance(kamodo_object.variables[varname]['xvec'], (dict)), 'kamodo_object.variables['+varname+"]['xvec']} is not a dictionary."

def test_variablecoorddict(kamodo_object, varname):
    #test that variable coordinate dictionary is not empty
    assert len(kamodo_object.variables[varname]['xvec'].keys())>0, f"{kamodo_object.variables[varname]['xvec']} is empty."
    
def test_nongriddedinterp(kamodo_object, varname, interp_values):
    #test interpolated value for a non-gridded function
    if '' in interp_values:
        idx = interp_values.index('')  #remove empty string from c3 for 2D+time functions
        interp_values.pop(idx)    
    for i in range(len(interp_values)): interp_values[i] = float(interp_values[i])
    if len(interp_values)==5:
        interp_result = kamodo_object[varname]((interp_values[0], interp_values[1], interp_values[2], interp_values[3]))
    elif len(interp_values)==4:
        interp_result = kamodo_object[varname]((interp_values[0], interp_values[1], interp_values[2]))
    else:
        assert interp_values[0]=='', f'Unexpected coordinate dimensionality encountered: {interp_values}'
    assert abs(interp_result-interp_values[-1]) < 0.000001 , f'expected result {interp_result} != stored result {interp_values[-1]}'
    
def test_griddedinterp(kamodo_object, varname, interp_values):
    #test interpolated value for a gridded function
    if '' in interp_values:
        idx = interp_values.index('')  #remove empty string from c3 for 2D+time functions
        interp_values.pop(idx)       
    for i in range(len(interp_values)): interp_values[i] = float(interp_values[i])
    if len(interp_values)==5:
        interp_result = kamodo_object[varname+'_ijk'](interp_values[0], interp_values[1], interp_values[2], interp_values[3])
    elif len(interp_values)==4:
        interp_result = kamodo_object[varname+'_ijk'](interp_values[0], interp_values[1], interp_values[2])
    else:
        assert interp_values[0]=='', f'Unexpected coordinate dimensionality encountered: {interp_values}'    
    assert abs(interp_result-interp_values[-1]) < 0.000001, f'expected result {interp_result} != stored result {interp_values[-1]}'

In [3]:
#loop through readers to test
from kamodo_ccmc.flythrough.model_wrapper import Choose_Model
for key in reader_dict.keys():
    #test generic functionality
    print(key, '\n----------------')
    print('Testing generic functionality...', end="")
    module = Choose_Model(key)  #import model-specific reader module
    test_all_variables(module)
    reader = module.MODEL()
    kamodo_object = reader(reader_dict[key][1], fulltime=False, variables_requested='all')  #reader_dict[key][1] = filename
    test_file_variables(kamodo_object)  
    test_file_varname(kamodo_object, reader_dict[key][3])  #reader_dict[key][3] = varname
    test_modelname(kamodo_object, key)
    print('tests passed.')
    
    #test different call behaviors
    print('Testing default calls...', end="")
    kamodo_object = reader(reader_dict[key][1])  #default
    test_nongriddedinterp(kamodo_object, reader_dict[key][3], reader_dict[key][8:13])  #reader_dict[key][8:13] = interp_values
    test_griddedinterp(kamodo_object, reader_dict[key][3], reader_dict[key][8:13])
    test_variablecoordinates(kamodo_object, reader_dict[key][3])
    test_variablecoorddict(kamodo_object, reader_dict[key][3])
    
    kamodo_object = reader(reader_dict[key][1], gridded_int=False) #default without gridded functions
    test_nongriddedinterp(kamodo_object, reader_dict[key][3], reader_dict[key][8:13])  #reader_dict[key][8:13] = interp_values
    test_variablecoordinates(kamodo_object, reader_dict[key][3])
    test_variablecoorddict(kamodo_object, reader_dict[key][3])
    print('tests passed.')
    
    print('Testing variable-specific calls...', end="")
    kamodo_object = reader(reader_dict[key][1], variables_requested=[reader_dict[key][3]])  #single function with gridded
    test_nongriddedinterp(kamodo_object, reader_dict[key][3], reader_dict[key][8:13])  #reader_dict[key][8:13] = interp_values
    test_griddedinterp(kamodo_object, reader_dict[key][3], reader_dict[key][8:13])
    
    kamodo_object = reader(reader_dict[key][1], variables_requested=[reader_dict[key][3]], gridded_int=False, 
                           runname=reader_dict[key][2])  #reader_dict[key][2] is the runname
    test_nongriddedinterp(kamodo_object, reader_dict[key][3], reader_dict[key][8:13])  #reader_dict[key][8:13] = interp_values
    print('tests passed.')
    
    print('Testing object attributes...', end="")
    test_variablecoordinates(kamodo_object, reader_dict[key][3])
    test_variablecoorddict(kamodo_object, reader_dict[key][3])
    test_utcstart(kamodo_object, reader_dict[key][4])  #reader_dict[key][4] = utc start timestamp
    test_utcstop(kamodo_object, reader_dict[key][5])  #reader_dict[key][5] = utc stop timestamp
    test_startstr(kamodo_object, reader_dict[key][6])  #reader_dict[key][6] = utc start time string
    test_stopstr(kamodo_object, reader_dict[key][7])  #reader_dict[key][7] = utc stop time string
    test_datetime(kamodo_object) 
    print('tests passsed.')
    print(f'All tests for {key} were successful.\n')

CTIPe 
----------------
Testing generic functionality...tests passed.
Testing default calls...tests passed.
Testing variable-specific calls...tests passed.
Testing object attributes...tests passsed.
All tests for CTIPe were successful.

IRI 
----------------
Testing generic functionality...tests passed.
Testing default calls...tests passed.
Testing variable-specific calls...tests passed.
Testing object attributes...tests passsed.
All tests for IRI were successful.

GITM 
----------------
Testing generic functionality...tests passed.
Testing default calls...tests passed.
Testing variable-specific calls...tests passed.
Testing object attributes...tests passsed.
All tests for GITM were successful.

SWMF_IE 
----------------
Testing generic functionality...tests passed.
Testing default calls...tests passed.
Testing variable-specific calls...tests passed.
Testing object attributes...tests passsed.
All tests for SWMF_IE were successful.

TIEGCM 
----------------
Testing generic functionality

c:\users\rringuet\kamodo_dev\kamodo\kamodo_ccmc\readers\tiegcm_4D.py:285: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  for key in gvar_list}  #store with key = standardized name


tests passed.
Testing variable-specific calls...tests passed.
Testing object attributes...tests passsed.
All tests for TIEGCM were successful.

OpenGGCM_GM 
----------------
Testing generic functionality...tests passed.
Testing default calls...tests passed.
Testing variable-specific calls...tests passed.
Testing object attributes...tests passsed.
All tests for OpenGGCM_GM were successful.

